In [40]:
def s(x): return " "*(10+x)
print(s(5)+".\n"+s(4)+"..:\n"+s(2)+"Hultnér\n"+s(0)+"Technologies\n\n@ahultner | https://hultner.se/")
import hypothesis
from hypothesis import given, assume, example
import hypothesis.strategies as st
import pytest
import ipytest
ipytest.config(rewrite_asserts=True, magics=True)
__file__ = "demo.ipynb"

               .
              ..:
            Hultnér
          Technologies

@ahultner | https://hultner.se/


#  Property based testing
## Case: SweBonk
In this demo I will use a pretend bank named **"SweBonk, _the_ bank"**, Bonk have some problems with their code which they need help with.

Let's start with a function which SweBonk is using for trading

In [9]:
from typing import Tuple, List, Optional

Price = float
# A Bonk customer will be prefixed with BNK and suffixed with a id e.g. BNK89
# Rival banks customers are prefixed RVL, eg RVL1209-3844-2534
Customer = str 
Order = Tuple[Customer, Price]

def match_buy(ticker: str, buyer: Order, sellers: List[Order]) -> Optional[Order]:
    """
    Bonk stock trade buy order matcher
    
    Args:
        ticker: The "ticker" id of stock to trade
        sellers: Sellers wanting to sell stocks at the pricer the buyer gave
    
    Returns: 
        Matched seller, None if we couldn't complete the trade
    """
    # First try to find internal customers so we don't have to trade with other bank
    matched_bonk_customer=((seller, price) for (seller, price) in sellers 
        if buyer[1] == price
         and seller[:3] == buyer[0][:3]
         # It's illegal to trade stocks with yourself
         and seller is not buyer[0]
    )
    
    try:
        match = next(matched_bonk_customer)
        return match
    except StopIteration:
        # No couldn't match a bonk customer, keep looking
        pass
        
    matched_customer=((seller, price) for (seller, price) in sellers 
        if buyer[1] == price and seller is not buyer[0]
    )
    try: 
        return next(matched_customer)
    except StopIteration:
        # Couldn't match any customer
        return None


SweBonk have been quite through in their testing, they have a 100% coverage and are confident in their solution. 
They've generalized the test method and are trying a large set of inputs known to trigger different edge cases.

In [13]:
%%run_pytest[clean] -qq --disable-pytest-warnings 

sellers = [
    # Illegal, will be skipped
    ("BNK421", 42.0),
    # Rival bank match, we'd rather want Bonk customers
    ("RVL1209-1321-2348", 42.20),
    ("RVL1209-9323-2148", 42.0),
    ("BNK12", 42.20),
    # This is the "best" match
    ("BNK01", 42.0),
]

def test_trade():
    verify_trade(buyer=("BNK421", 42.0))
    verify_trade(buyer=("BNK421", 42.2))
    verify_trade(buyer=("BNK12", 42.2))
    verify_trade(buyer=("BNK01", 42.2))
    verify_trade(buyer=("BNK123", 42.2))
    verify_trade(buyer=("BNK123", 42.2))
    verify_trade(buyer=("BNK123", 40))
    

def verify_trade(buyer):
    winner = match_buy("APL", buyer, sellers)
    # If there's no winner
    if winner is None:
        for (seller, price) in sellers:
            # Then assert at no seller price match
            assert price != buyer[1]
        return
    
    # A winner was found, continue        
    # Not traded with yourself
    assert winner[0] != buyer[0]
    # Price is a match
    assert winner[1] == buyer[1]
    
    # If winner is from another bank, assert it could match internally
    if winner[0][:3] != buyer[0][:3]:
        for seller in sellers:
            if buyer[1] == seller[1] and buyer != seller:
                assert seller[0][:3] != buyer[0][:3]


.                                                                                                                                                      [100%]


Everything looks fine and tests are passing but SweBonk are still having problems with illegal trades being made on their platform. They can't figure out why but have hired Hultnér Technologies to look if there's room for improvment in their testing.

In [14]:
%%run_pytest[clean] -qq --disable-pytest-warnings 

customers = st.text(min_size=4)
orders = st.tuples(customers, st.floats(min_value=0.1))

@given(
    orders,
    st.lists(orders),
)
def test_trade(buyer, sellers):
    winner = match_buy("APL", buyer, sellers)
    assume(winner is not None)
    # Not traded with yourself
    assert winner[0] != buyer[0]
    # Price is a match
    assert winner[1] == buyer[1]
    
    # If winner is from another bank, assert it could match internally
    if winner[0][:3] != buyer[0][:3]:
        for seller in sellers:
            if buyer[1] == seller[1] and buyer != seller:
                assert seller[0][:3] != buyer[0][:3]



F                                                                                                                                                      [100%]
========================================================================== FAILURES ==========================================================================
_________________________________________________________________________ test_trade _________________________________________________________________________

    @given(
>       orders,
        st.lists(orders),
    )
    def test_trade(buyer, sellers):

<ipython-input-14-34bb1f303cde>:6: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

buyer = ('0000', 0.1), sellers = [('0000', 0.1)]

    @given(
        orders,
        st.lists(orders),
    )
    def test_trade(buyer, sellers):
        winner = match_buy("APL", buyer, sellers)
        assume(winner is not None)

Turns out that `match_buy` sometimes let's through invalid buys

In [32]:
a = "BNK!"
b = "BNK!"
a is b

False

This is becuase of the following statement:
```python
seller is not buyer[0]
``` 
The `is`-operator checks for __identinty, not equality__.   
In the case for short and simple strings cpython stores them as the same object in memory hence the code worked until SweBonk started to get more customers and their customer id´s got to long to share the same space in memory. At this point they started letting through illegal trades.

Let's create an updated method and run the tests again.

In [34]:
%%run_pytest[clean] -qq --disable-pytest-warnings 
# Updated match buy and rerun previous tests
def match_buy_2(ticker: str, buyer: Order, sellers: List[Order]) -> Optional[Order]:
    """
    Bonk stock trade buy order matcher
    …
    """
    # First try to find internal customers so we don't have to trade with other bank
    matched_bonk_customer=((seller, price) for (seller, price) in sellers 
        if buyer[1] == price
         and seller[:3] == buyer[0][:3]
         # It's illegal to trade stocks with yourself
         and seller != buyer[0] # <---<< Fixed here
    )
    
    try:
        match = next(matched_bonk_customer)
        return match
    except StopIteration:
        # No couldn't match a bonk customer, keep looking
        pass
        
    matched_customer=((seller, price) for (seller, price) in sellers 
        if buyer[1] == price and seller != buyer[0] # <---<< And here
    )
    try: 
        return next(matched_customer)
    except StopIteration:
        # Couldn't match any customer
        return None

@given(
    orders,
    st.lists(orders),
)
def test_trade(buyer, sellers):
    winner = match_buy_2("APL", buyer, sellers)
    assume(winner is not None)
    # Not traded with yourself
    assert winner[0] != buyer[0]
    # Price is a match
    assert winner[1] == buyer[1]
    
    # If winner is from another bank, assert it could match internally
    if winner[0][:3] != buyer[0][:3]:
        for seller in sellers:
            if buyer[1] == seller[1] and buyer != seller:
                assert seller[0][:3] != buyer[0][:3]

.                                                                                                                                                      [100%]


Based on this knowledge the SweBonk engineers took their learnings and applied on another part of there business where illegal things were happning even though their code were, they had been confident in that their solution were sufficient but had as of recently been disproven. 

Namley their **money laundering protection** software.

In [54]:
"""
SweBonk – Money Laundering Protection
TOP SECRET: Enterprise software, only for personel with security clearence
"""
from typing import Tuple, List

Amount = float
Account = str
Customer = str
# (From, Amount, Target)
Transaction = Tuple[Account, Amount, Customer]

# Current production code 
# DON'T CHANGE, CRITICAL INFRASTRUCTURE
def verify_transaction(incoming: Transaction, blacklist: List[Account]) -> bool:
    """ return false if incoming Transaction is from blacklisted account
    
    FOR SWEBONK EYES ONLY
    
    Args:
        incoming: Transaction from external bank
        blacklist: List of known accounts used for money laundering
    
    Returns:
        True if transaction is safe, False if it's from an invalid account
    """
    (account, *_) = incoming
    for bad_account in blacklist:
        if account is bad_account:
            # Account is blacklisted
            return False
    # Not found in blacklist, transfer is illegal
    return True


In [55]:
%%run_pytest[clean] -qq --disable-pytest-warnings 

transaction = st.tuples(st.text(), st.floats(), st.text())

@given(
    transaction,
    st.lists(st.text()),
)
def test_blacklist(transfer, blacklist):
    (account, *_) = transfer
    legal = verify_transaction(transfer, blacklist)
    if legal:
        for banned in blacklist:
            assert account != banned

F                                                                                                                                                      [100%]
========================================================================== FAILURES ==========================================================================
_______________________________________________________________________ test_blacklist _______________________________________________________________________

    @given(
>       transaction,
        st.lists(st.text()),
    )
    def test_blacklist(transfer, blacklist):

<ipython-input-55-8e86a8973405>:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

transfer = ('Ā', 0.0, ''), blacklist = ['Ā']

    @given(
        transaction,
        st.lists(st.text()),
    )
    def test_blacklist(transfer, blacklist):
        (account, *_) = transfer
        legal = verif

In [56]:
%%run_pytest[clean] -qq --disable-pytest-warnings 

# New and improved version, fixed dangerous bug
def verify_transaction_2(incoming: Transaction, blacklist: List[Account]) -> bool:
    """ … """
    (account, *_) = incoming
    return account not in blacklist

@given(
    transaction,
    st.lists(st.text()),
)
def test_blacklist(transfer, blacklist):
    (account, *_) = transfer
    legal = verify_transaction_2(transfer, blacklist)
    if legal:
        for banned in blacklist:
            assert account != banned

.                                                                                                                                                      [100%]


And there we have it, the bug is fixed and transfers from blacklisted accounts are no longer slipping through!







---


```
  .  Hultnér      
 ..: Technologies 
```
https://hultner.se  
contact@hultner.se  
https://twitter.com/ahultner

